<a href="https://colab.research.google.com/github/DeisyZermeno/desihigh/blob/main/Accesing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sparclclient

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.8/113.8 kB 3.5 MB/s eta 0:00:00


In [ ]:
from astropy.utils.data import download_file # https://learn.astropy.org/tutorials/FITS-images.html
from astropy.io import fits
import numpy as np
import pandas as pd
from sparcl.client import SparclClient

Ahora descargaremos el [catálogo](https://data.desi.lbl.gov/public/edr/spectro/redux/fuji/zcatalog/zall-pix-fuji.fits) atravez de un url.

In [ ]:
url = "https://data.desi.lbl.gov/public/edr/spectro/redux/fuji/zcatalog/zall-pix-fuji.fits"
# Descarga el archivo desde la URL
zall_pix_file = download_file(url, cache=True)

In [ ]:
zall_data = fits.getdata(zall_pix_file)

Veamos que tipos de espectros podemos encontrar en el catálogo

In [ ]:
np.unique(zall_data['SPECTYPE'])

chararray(['GALAXY', 'QSO', 'STAR'], dtype='<U6')

Excelente ahora puedes obtener espectros de galaxias, estrellas y cuasares, vamos a ello.

In [ ]:
typ = zall_data['SPECTYPE']
galaxies = zall_data[zall_data["SPECTYPE"] == "GALAXY"]
stars = zall_data[zall_data["SPECTYPE"] == "STAR"]
qso = zall_data[zall_data["SPECTYPE"] == "QSO"]

In [ ]:
# Esta funcion nos ayuda a elegir espectros random de todo el catálogo.
def get_random_targetids(arr, n):
  random_indices = np.random.choice(len(arr), size=n, replace=False)
  return arr[random_indices]["TARGETID"]

Ahora definamos cuantos espectros quieres obtener:

In [ ]:
# Numero de espectros
N = 5

Veamos, obtengamo los TARGETID de 5 espectros de cadas tipo.

In [ ]:
# NOTA: Aun estoy mejorando el código, la idea es que los TARGETID se regresen al usario ya en una lista, para que no los tenga que copiar y pegar.
print("Gaxalies")
random_galaxies_targetids = get_random_targetids(galaxies, N)
for r in random_galaxies_targetids:
  print(r)

Gaxalies
616088527960015309
616089185530413103
616088570297320320
39627847651823912
39627770275302015


In [ ]:
print("Stars")
random_stars_targetids = get_random_targetids(stars, N)
for r in random_stars_targetids:
  print(r)

Stars
39627586527036018
39633154046167459
39627764193559286
-810679429
39633426923391769


In [ ]:
print("QSO")
random_qso_targetids = get_random_targetids(qso, N)
for r in random_qso_targetids:
  print(r)

QSO
39633282647721587
103625124741152
39633405603745713
39633293380946186
39627920989229166


Si quieres obtener la información de dicho espectro, hagamos lo siguiente. Digamos que quieres obtener la informacion de wavelength, flux y redshift de los espectros de cuasares que obtruviste, entonces haremos uso de **client.retrieve_by_specid** al que le pasaremos como parámetros la lista de TARGETID de los espectros que obtuviste y la información que quieres de ellos.

In [ ]:
# Necesitamos generar un "cliente" para poder interactuar con los servicios de SPARCL
# Una vez hecho esto podremos acceder a datos y realizar operaciones con ellos!!
client = SparclClient()

In [ ]:
TRGETID=[39633282647721587, 103625124741152,39633405603745713,39633293380946186,39627920989229166]
inf = ['wavelength', 'flux', 'redshift']
spectra = client.retrieve_by_specid(TRGETID,
    include = inf)
spec = spectra.records

In [ ]:
df = pd.DataFrame(spec)
df

,redshift,flux,wavelength,_dr
0,1.425212,"[4.910739898681641, 3.4554779529571533, 4.4037...","[3600.0, 3600.8, 3601.6000000000004, 3602.4000...",DESI-EDR
1,0.894964,"[16.631006240844727, 19.67969512939453, 10.660...","[3600.0, 3600.8, 3601.6000000000004, 3602.4000...",DESI-EDR
2,1.423316,"[0.09027601033449173, 1.7164666652679443, 0.37...","[3600.0, 3600.8, 3601.6000000000004, 3602.4000...",DESI-EDR
3,1.789569,"[0.6580186486244202, -1.9306870698928833, 1.67...","[3600.0, 3600.8, 3601.6000000000004, 3602.4000...",DESI-EDR
4,1.715298,"[2.0987820625305176, 0.8841259479522705, 3.887...","[3600.0, 3600.8, 3601.6000000000004, 3602.4000...",DESI-EDR
5,0.215073,"[17.903379440307617, -4.437004089355469, -21.7...","[3600.0, 3600.8, 3601.6000000000004, 3602.4000...",DESI-EDR
